In [1]:
import toolbox as tb
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [7]:
movies_df = pd.read_csv("../wth_actors_name/tools/data_preparation/new_movies_scraped.csv")

### Clean Actors

In [8]:
actors = list(movies_df["actors"])

In [9]:
for i, row in enumerate(actors):
    if type(row) != float:
        actors[i] = row.replace("[", "").replace("]", "").replace("'", "").replace(", ", ",")

### Clean Awards

In [10]:
awards = list(movies_df["awards"])

In [11]:
for i, row in enumerate(awards):
    if type(row) != float:
        row = row.replace(" ", "").replace("\\n", "").replace("[", "").replace("]", "")
        awards[i] = row.replace("'", "").replace("for", "").replace("&", "").replace("Another", "").replace(".", "")

### Clean Country

In [12]:
countries = list(movies_df["country"])

In [13]:
for i, row in enumerate(countries):
    if type(row) != float:
        countries[i] = row.replace("\n", "").replace("|", ",").replace(" ", "")

### Clean  Language

In [14]:
languages = list(movies_df["language"])

In [15]:
for i, row in enumerate(languages):
    if type(row) != float:
        languages[i] = row.replace("\n", "").replace("|", ",").replace(" ", "")

### Clean Worldwide Gross

In [16]:
ww_gross = list(movies_df["worldwideGross"])

In [17]:
for i, row in enumerate(ww_gross):
    if type(row) != float:
        ww_gross[i] = row.replace(" ", "").replace("\n", "")


### Replacing dirty with clean strings

In [18]:
movies_df["actors"] = actors
movies_df["awards"] = awards
movies_df["country"] = countries
movies_df["language"] = languages
movies_df["worldwideGross"] = ww_gross

### Separate awards string

In [20]:
awards = list(movies_df["awards"])

In [21]:
for i, row in enumerate(awards):
    if type(row) != float:
        awards[i] = row.split(",")

In [22]:
n_wins_list = []
n_noms_list = []
oscar_wins_list = []
oscar_noms_list = []
gg_wins_list = []
gg_noms_list = []

for i, row in enumerate(awards):
    
    n_wins = "0"
    n_noms = "0"
    oscar_wins = "0"
    oscar_noms = "0"
    gg_wins = "0"
    gg_noms = "0"

    if type(row) != float:

        if len(row) == 1:
            
            
            if row[0].find("Oscar") != -1:
                if row[0].find("Won") != -1:
                    oscars_i = row[0].find("Oscar")
                    oscar_wins = row[0][len("Won"):oscars_i]

                if row[0].find("Nominated") != -1:
                    oscars_i = row[0].find("Oscar")
                    oscar_noms = row[0][len("Nominated"):oscars_i]


            elif row[0].find("GoldenGlobe") != -1:
                if row[0].find("Won") != -1:
                    gg_i = row[0].find("GoldenGlobe")
                    gg_wins = row[0][len("Won"):gg_i]

                if row[0].find("Nominated") != -1:
                    gg_i = row[0].find("GoldeGlobe")
                    gg_noms = row[0][len("Nominated"):gg_i]
            
            else:   
                wins_i = row[0].find("win")

                if wins_i != -1:
                    n_wins = row[0][:wins_i]
                #row[1] = row[1][wins_i + len("wins"):]

                noms_i = row[0].find("nomination")

                if row[0].find("wins") != -1:
                    n_noms = row[0][wins_i + len("wins"):noms_i]
                else:
                    if wins_i == -1:
                        n_noms = row[0][:noms_i]
                    else:
                        n_noms = row[0][wins_i + len("win"):noms_i]



        # Get wins and nominations if len(row) > 1
        if len(row) > 1:

            if row[0].find("Oscar") != -1:
                if row[0].find("Won") != -1:
                    oscars_i = row[0].find("Oscar")
                    oscar_wins = row[0][len("Won"):oscars_i]

                if row[0].find("Nominated") != -1:
                    oscars_i = row[0].find("Oscar")
                    oscar_noms = row[0][len("Nominated"):oscars_i]


            if row[0].find("GoldenGlobe") != -1:
                if row[0].find("Won") != -1:
                    gg_i = row[0].find("GoldenGlobe")
                    gg_wins = row[0][len("Won"):gg_i]

                if row[0].find("Nominated") != -1:
                    gg_i = row[0].find("GoldenGlobe")
                    gg_noms = row[0][len("Nominated"):gg_i]        


            wins_i = row[1].find("win")
            
            if wins_i != -1:
                n_wins = row[1][:wins_i]
            #row[1] = row[1][wins_i + len("wins"):]

            noms_i = row[1].find("nomination")

            if row[1].find("wins") != -1:
                n_noms = row[1][wins_i + len("wins"):noms_i]
            else:
                if wins_i == -1:
                    n_noms = row[1][:noms_i]
                else:
                    n_noms = row[1][wins_i + len("win"):noms_i]
    
    n_wins_list.append(n_wins)
    n_noms_list.append(n_noms)
    oscar_wins_list.append(oscar_wins)
    oscar_noms_list.append(oscar_noms)
    gg_wins_list.append(gg_wins)
    gg_noms_list.append(gg_noms)

In [23]:
movies_df["oscarsWon"] = oscar_wins_list
movies_df["oscarsNom"] = oscar_noms_list
movies_df["goldenGlobesWon"] = gg_wins_list
movies_df["goldenGlobesNom"] = gg_noms_list
movies_df["totalWins"] = n_wins_list
movies_df["totalNoms"] = n_noms_list

In [25]:
movies_df = movies_df[movies_df["numVotes"].notnull()]
movies_df = movies_df[movies_df["runtimeMinutes"] != "\\N"]
movies_df["totalNoms"] = movies_df["totalNoms"].fillna(0)
movies_df[["totalNoms"]] = movies_df[["totalNoms"]].replace(to_replace="", value="0")

In [39]:
movies_df["numVotes"] = movies_df["numVotes"].astype(int)

movies_df["runtimeMinutes"] = movies_df["runtimeMinutes"].astype(int)

movies_df["totalNoms"] = movies_df["totalNoms"].astype(int)